In [2]:
import os

import numpy as np
import pandas as pd

from PIL import Image
from matplotlib import pyplot as plt

import torch
import torchvision
import torchvision.transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, -1)

In [18]:
Net = Network()
optimizer = optim.Adagrad(Net.parameters(), lr=0.01)
loss_fun = nn.CrossEntropyLoss()

In [19]:
if torch.cuda.is_available():
    Net = Net.cuda()
    loss_fun = loss_fun.cuda()